In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from evaluation import evaluate
from problem_instance.point_cloud_registeration import model
from pc_formulations import *
import time
import open3d as o3d

In [2]:
cloud = o3d.io.read_point_cloud("bunny.pcd")
if cloud.is_empty(): exit()

In [3]:
cloud.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1, max_nn=30))

In [4]:
points = 100 * np.asarray(cloud.points)

In [5]:
num_points = 30
mu, sigma = 0, 0.1 # mean and standard deviation
beta = np.sqrt(3*(5.54*sigma)**2)
s1 = np.random.normal(mu, sigma, num_points)
s2 = np.random.normal(mu, sigma, num_points)
s3 = np.random.normal(mu, sigma, num_points)
sqrt2d2 = np.sqrt(2)/2
R = np.array([[sqrt2d2, sqrt2d2, 0], [-sqrt2d2, sqrt2d2, 0], [0, 0, 1]])
t = np.array([0.5, 0.5, 0.3])
noise = np.hstack((s1.reshape(-1, 1), s2.reshape(-1, 1), s3.reshape(-1, 1)))

In [12]:
num_data = 5
outrates = np.arange(0.1, 1, 0.1)
num_outliers = (num_points * outrates).astype(int)
n = len(points)

In [13]:
def run(num_data, method, solver, api_key=None):

  overall_acc = np.zeros(len(outrates))
  in_pre = np.zeros(len(outrates))
  out_rej = np.zeros(len(outrates))

  
  for i in range(num_data):
    currdata_ind = np.random.choice(np.arange(n), num_points, replace=False)
    currdata = points[currdata_ind]
    new_coord = model(R, t, currdata, noise)

    for j, num_outlier in enumerate(num_outliers):
      outliers = 1 * np.random.rand(num_outlier, 3) # outliers points in 3D
      outliers_cord_t = model(R, t, outliers, np.zeros((num_outlier, 3)))
      new_coord_with_outliers = np.vstack((new_coord[:-num_outlier], outliers_cord_t))
      inlier_mask = np.zeros(num_points, dtype=bool)
      inlier_mask[:-num_outlier] = True
      perm = np.random.permutation(num_points)
      new_coord_with_outliers = new_coord_with_outliers[perm]
      p_coordinates = currdata[perm]
      inlier_perm_mask = inlier_mask[perm]
      try:
        start = time.time()
        case1f_sol = method(p_coordinates, new_coord_with_outliers, beta, solver, api_key)
        runtime += time.time() - start
      except ValueError:
        print("ValueError during {}, {}".format(i, j))
        continue
      acc, ipr, orr = evaluate(num_points, inlier_perm_mask, case1f_sol, display=False)
      overall_acc[j] += acc
      in_pre[j] += ipr
      out_rej[j] += orr
  return overall_acc / num_data, in_pre / num_data, out_rej / num_data

In [ ]:
# How to use the run function:
run(10, case1f, "dwave_annealing")
run(10, case1f, "gurobi")
run(10, case1f, "dwave_quantum")